### Extract Lyrics Embeddings

In [4]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import scipy as sp

/Users/chloe/opt/miniconda3/envs/BKMS/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# import dataset
data = pd.read_csv('./spotify_data.csv', index_col=[0])
lyrics_data = pd.DataFrame(data['lyrics'])
lyrics_data['embedding'] = ''
lyrics_data['lyrics'] = lyrics_data['lyrics'].apply(lambda e: e.replace('"',''))

In [7]:
# define model
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, pipeline
import torch

config = RobertaConfig.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
config.output_hidden_states = True
tok = RobertaTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
model = RobertaModel.from_pretrained("j-hartmann/emotion-english-distilroberta-base", config=config)

# extract embedding
lyrics_data['embedding'] = lyrics_data['lyrics'].apply(lambda e: np.array(model(torch.tensor([tok.encode(e)[:512]])).pooler_output.squeeze().detach()))

# lyrics_data['embedding'] = lyrics_data['lyrics'].apply(lambda e: model(torch.tensor([tok.encode(e)])).pooler_output.squeeze())
lyrics_data

Some weights of RobertaModel were not initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (674 > 512). Running this sequence through the model will result in indexing errors


,lyrics,embedding
160,Aah Aah The games you played were never fun Yo...,"[-0.13932583, 0.12747765, 0.2656628, 0.2385701..."
445,The lights go out and I can't be saved Tides t...,"[-0.4183085, 0.15299061, 0.048193373, 0.524825..."
763,"'Cause you're a sky, 'cause you're a sky full ...","[0.0012021133, -0.038288224, -0.2541366, 0.362..."
856,I will not make the same mistakes that you did...,"[-0.2938952, -0.06864088, 0.14853874, 0.564785..."
1169,Heyy Heeey Heey Your lipstick stains On the fr...,"[0.0226809, 0.05377083, -0.027528005, 0.462929..."
...,...,...
16203,"Yeah, you! Yeah, you! I used to wanna be Livin...","[-0.16977464, 0.10209923, -0.25834057, 0.28879..."
16284,"If you don't wanna see me Did a full 180, craz...","[-0.43178084, -0.4026616, -0.07043896, 0.31529..."
16771,"Nah, nah, nah Cake by the ocean Oh, no See you...","[-0.14135475, 0.09500975, -0.38598028, 0.28835..."
17046,"Come on, come on, turn the radio on It's Frida...","[-0.12562634, -0.2382564, -0.23705368, 0.41674..."


In [11]:
lyrics_data['embedding'].iloc[0].shape

(768,)

In [5]:
import pickle
with open('./lyrics_embedding.pkl', 'wb') as f:
    pickle.dump(lyrics_data, f)